En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Load common libraries and file path

In [2]:
file_path = "../resources/farmers-protest-tweets-2021-2-4.json"
import matplotlib.pyplot as plt
import memory_profiler
from cProfile import Profile
from pstats import Stats, SortKey

def plot(x_values, y_values, x_label, y_label, title=''):
    plt.plot(x_values, y_values, marker='o')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid(True)
    plt.show()

For the first exercise q1_memory,  I've implemented two approaches.  Lets run them in two different cells and decide what is the best


<h5>1. First Approach:</h5>
- Retrieve the top 10 dates. </br>
- Filter users who posted on those days. </br>
- For each day, identify the top user who posted on that day. </br>
Based on the results, this approach proves superior as it exclusively fetches the required dates while disregarding the remainder. 

In [ ]:
from q1_memory import q1_memory
%load_ext memory_profiler
%mprun -f q1_memory_v2 q1_memory(file_path)

In [ ]:
from q1_memory import q1_memory
memory_usage = memory_profiler.memory_usage((q1_memory, (), {'file_path': file_path}))
x_values = list(range(1, len(memory_usage) + 1))  
# Plot the memory usage over time
plot(x_values, memory_usage, 'Time', 'Memory usage (MiB)', 'Memory Usage Over Time')

<h5>2. Second approach</h5>
- Get a list with all tuples (date, username)
- Get top 10 of posted dates
- Get top 1 user who posted in each day

In [ ]:
from typing import List, Tuple
from datetime import datetime
import json
from collections import Counter

def q1_memory_v2(file_path: str) -> List[Tuple[datetime.date, str]]:
    #Due to I need to minimize memory space, generator is a good approach
    def read_json():
        with open(file_path, 'r') as file:
            while line := file.readline():
                yield json.loads(line)

    items = read_json()
    date_user = []
    #get date and user for every item and append to a list
    for item in items:
        only_date = datetime.strptime(item['date'], "%Y-%m-%dT%H:%M:%S%z").date()
        date_user.append((only_date, item['user']['username']))

    #get the top 10 dates
    date_counts = Counter(date for date, _ in date_user).most_common(10)
    dates = [d for d, _ in date_counts]
    del date_counts
    result = []
    # for each top date get the top user. Append to the final result the date and user
    for d in dates:
        user_by_date = Counter(user for date, user in date_user if date == d).most_common(1)
        result.append((d, user_by_date[0][0]))

    return result

memory_usage = memory_profiler.memory_usage((q1_memory_v2, (), {'file_path': file_path}))
x_values = list(range(1, len(memory_usage) + 1))  
# Plot the memory usage over time
plot(x_values, memory_usage, 'Time', 'Memory usage (MiB)', 'Memory Usage Over Time')

As a conclusion, the <b>First approach</b> is the optimal approach

<h4>Exercise 1 Time performance:</h4> 
This time, I've opted to utilize <b>Pandas</b> due to its utilization of <b>NumPy</b> under the hood. The NumPy library, partially written in C, significantly enhances performance compared to native Python, often by orders of magnitude.
The following cell employs cProfile and pstats to execute q1_time and measure its execution time. I've chosen to display only the 15 functions with the longest execution times.

In [ ]:
from q1_time import q1_time

with Profile() as profile:
    print(q1_time(file_path))
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats(15)
    )

<h4>Exercise 2: Memory problem </h4>
To solve this exercise I've decided to use to external libraries emoji and regex. The last allows to find grapheme clusters occurrences

In [ ]:
from q2_memory import q2_memory
%reload_ext memory_profiler
%mprun -f q2_memory q2_memory(file_path)

In [ ]:
from q2_memory import q2_memory
memory_usage = memory_profiler.memory_usage((q2_memory, (), {'file_path': file_path}))
x_values = list(range(1, len(memory_usage) + 1))  
# Plot the memory usage over time
plot(x_values, memory_usage, 'Time' ,'Memory usage (MiB)', 'Memory Usage Over Time')

<h4>Exercise 2: Time problem. </h4> 
Let see which approach is faster to load the file


In [ ]:
import pandas as pd
with Profile() as profile:
    df = pd.read_json(file_path, lines=True)
    print(df.head())
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats(15)
    )

Now let use readlines to process the file with native Python

In [ ]:
import json

with Profile() as profile:
    file = open(file_path, 'r')
    data = json.loads(f"[{','.join(file.readlines())}]")
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats(15)
    )

As a conclusion for the file load time there is no difference

A first approach that I've tried to use was with Pandas. But this solution is suboptimal compared with the implemented in q2_time similar to q2_memory but loading the complete file instead to use generator

In [ ]:
from typing import List, Tuple
import pandas as pd
import emoji
import regex

#Subotpimal solution
def q2_time_v2(file_path: str) -> List[Tuple[str, int]]:
    df = pd.read_json(file_path, lines=True)
    # Get a list of emojis in a given text
    df['emojis'] = df['renderedContent'].apply(
                                lambda x: [c for c in regex.findall(r'\X', x) if emoji.is_emoji(c)])

    #df = df.loc[:, ~df.columns.isin(['emojis'])]
    # I need to explode that emoji list
    exploded_df = df.explode('emojis')
    # Count emojis occurrence
    exploded_df = exploded_df.groupby('emojis').size().reset_index(name='count_emojis')
    # Keep the top 10 used emojis
    exploded_df = exploded_df.nlargest(n=10, columns='count_emojis', keep='all')

    return list(exploded_df[['emojis', 'count_emojis']].to_records(index=False))
    
with Profile() as profile:
    print(q2_time_v2(file_path))
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats(15)
    )


This solution improves the execution time compared with the use of pandas

In [ ]:
from q2_time import q2_time

with Profile() as profile:
    print(q2_time(file_path))
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats(15)
    )

<h4>Exercise 3: Top 10 influencer users </h4>
I'll use the library jsonpath-ng in order to make easy get all username values from <b>$.mentionedUsers[*].username</b>

<h5>1. Memory Solution:</h5>
I.  Read line by line using iterator, but returns the list of mentioned users in the tweet</br>
II. Sum new occurrences to the previous result 


In [ ]:
from q3_memory import q3_memory
%reload_ext memory_profiler
%mprun -f q3_memory q3_memory(file_path)

In [ ]:
from q3_memory import q3_memory
memory_usage = memory_profiler.memory_usage((q3_memory, (), {'file_path': file_path}))
x_values = list(range(1, len(memory_usage) + 1))  
# Plot the memory usage over time
plot(x_values, memory_usage, 'Time' ,'Memory usage (MiB)', 'Memory Usage Over Time')

Lets check time using <b>orjson</b>

In [ ]:
import orjson

def read_json():
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data_json = '[' + ','.join(lines) + ']'
        return orjson.loads(data_json)

        
def run_exec():
    items = read_json()
    
with Profile() as profile:
    run_exec()
    (
    Stats(profile).strip_dirs().sort_stats(SortKey.TIME).print_stats()
    )

In [3]:
from q3_memory import q3_memory

with Profile() as profile:
    print(q3_memory(file_path))
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats()
     .dump_stats('q3_memory_time_measure.prof')
    )

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
         6662043 function calls (6544068 primitive calls) in 99.057 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117408   21.637    0.000   64.995    0.001 q3_memory.py:14(read_json)
   117407   21.274    0.000   21.274    0.000 q3_memory.py:18(<listcomp>)
   117407   19.223    0.000   19.327    0.000 q3_memory.py:25(<dictcomp>)
        1   14.446   14.446   99.035   99.035 q3_memory.py:9(q3_memory)
   117407    4.167    0.000    4.167    0.000 decoder.py:343(raw_decode)
   220810    2.251    0.000    8.155    0.000 jsonpath.py:585(_find_base)
   234814    1.901    0.000    7.295    0.000 jsonpath.py:267(<listcomp>)
   220810    1.539    0.000    3.000    0.000 jsonpath.py:554(get_field_d

In [4]:
from q3_time import q3_time

with Profile() as profile:
    print(q3_time(file_path))
    (
     Stats(profile)
     .strip_dirs()
     .sort_stats(SortKey.TIME)
     .print_stats()
     .dump_stats('q3_time_time_measure.prof')
    )

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
         4842938 function calls (4842714 primitive calls) in 32.553 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    6.132    6.132    6.132    6.132 q3_time.py:21(<listcomp>)
        1    4.574    4.574    5.569    5.569 {orjson.loads}
   441621    4.473    0.000    5.414    0.000 jsonpath.py:102(__init__)
        4    2.773    0.693   18.723    4.681 jsonpath.py:267(<listcomp>)
   220810    2.670    0.000   10.785    0.000 jsonpath.py:585(_find_base)
   220810    1.845    0.000    5.252    0.000 jsonpath.py:554(get_field_datum)
   779838    1.538    0.000    1.622    0.000 jsonpath.py:95(wrap)
   220810    1.237    0.000    6.489    0.000 jsonpath.py:587(<listcomp>)
     3262    0

<h4>Exercise 3 Time and Memory Analysis</h4>
<h5>Memory Analysis</h5>
Both Files q3_memory.py and q3_time.py imports `memory_profiler` in order to check the consumed memory. If you want to see how much memory use each one, follow these steps: (eg. q3_memory.py)</br>
1. Uncomment these lines at the end of the file </br>
```python 
# path = '../resources/farmers-protest-tweets-2021-2-4.json'
# q3_memory(path)
```
2. From the terminal run this `$ mprof run python q3_memory.py`
3. This will generate a file called `q3_memory.log` in which you can see the evolution of memory usage

</br></br>
<h5>Time Analysis</h5>
For time Analysis I've decided to use cProfile, Stats along side with Tuna package (to plot). As you can see in The last two previous cells, there are `dump_stats` which generate two files to see the profiling for each file. </br>
Follow these steps: </br>
1. Run a given cell. This will generate the `.prof` file </br>
2. In a console run `tuna q3_memory_time_measure.prof` This will open the browsert showing the profiling

<h4>AWS CloudFormation</h4>
Finally, if you want to deploy and test in AWS environment follow this steps:
1. Upload the zip file to S3 bucket
2. Edit template.yaml and replace the line `aws s3 cp s3://dea-c01-marcos/data-eng-latam/tweets.json.zip .` with your actual file path in s3. IMPORTANT: upload the zip file
3. Do the deploy `aws cloudformation deploy --stack-name comandrunner-data-eng-challenge --template-file ./template.yaml` </br>
This cloudFormation use AWSUtility::CloudFormation::CommandRunner defined here https://github.com/aws-cloudformation/aws-cloudformation-resource-providers-awsutilities-commandrunner which allows execute bash commands in EC2.</br>
This deploy will run the commands detailed in <b>RunCommand</b> which will run `python app.py` which is also in the root and will get the stats for both Memory consuming and time executing 